In [4]:
%run ./ArenaDataLib.ipynb

import pathlib
path = pathlib.Path().resolve().as_posix()


In [9]:
def parseArenaToCSV(LOCATIONS,GAME,GENERATION,STARTERS,LEADER,ARENA):
    pathToSaveCSV = f"{path}/data/{GAME}/{ARENA}"
    arenaData =  gatherDataForArena(LOCATIONS,GAME,GENERATION,STARTERS,LEADER)
    
    # EXTRACT AREADATA TO DataFrame
    areasData = []
    areaDataColumnNames = ['name','steps','averageEncounters','encounterablePokemons']

    areas = arenaData['areas']
    for area in areas:
        areaData = [area.name, area.steps, area.averageEncounters, area.encounterablePokemons]
        areasData.append(areaData)
    areaTable =  pd.DataFrame(areasData, columns = areaDataColumnNames)
    # ----- #
    
    # EXTRACT ENCOUNTERABLEPOKEMON / MyPokemon DataFrame
    myPokemons = []
    myPokemonColumnNames = ['name', 'moves', 'encounterOrEvolutionTrigger']
    
    pokemons = arenaData['catchablePokemons']
    for pokemonKey in pokemons:
        pokeValue = pokemons[pokemonKey]
        moves = []
        for move in pokeValue.moves:
            moves.append({'name':move[0]['name'],'levelLearnedAt':move[1]})
        myPokemon = [pokeValue.name, moves,pokeValue.encounterOrEvolutionTrigger ]
        myPokemons.append(myPokemon)
    myPokemonTable =  pd.DataFrame(myPokemons, columns = myPokemonColumnNames)
    # ----- #
    
    # EXTRACT MOVESOFINTEREST DataFrame
    # extracts moves from encounterablePokemon and LeaderPokemon
    movesOfInterest = []
    movesOfInterestColumNames = ['name', 'moveType', 'damageClass', 'power', 'pp', 'priority', 'accuracy']
    moveSet = {}
    pokemons = arenaData['catchablePokemons']
    for pokemonKey in pokemons:
        pokeValue = pokemons[pokemonKey]
        pokeMoves = []
        for move in pokeValue.moves:
            moveData = move[0]
            moveSet[moveData['name']] = [moveData['name'],moveData['moveType'],moveData['damageClass'],moveData['power'],moveData['pp'],moveData['priority'],moveData['accuracy']]
    for pokemon in arenaData['arenaLeader'].team:
        leaderPokeMoves = pokemon[0].moves
        for move in leaderPokeMoves:
            moveData = move
            moveSet[moveData['name']] = [moveData['name'],moveData['moveType'],moveData['damageClass'],moveData['power'],moveData['pp'],moveData['priority'],moveData['accuracy']]
    for moveKey in moveSet:
        move = moveSet[moveKey]
        movesOfInterest.append(move)
    movesOfInterestTable =  pd.DataFrame(movesOfInterest, columns = movesOfInterestColumNames)
    # ----- #
    
    # EXTRACT ArenaLeader DataFrame
    ArenaLeader = []
    ArenaLeaderColumnNames = ['name', 'averageLevel', 'pokemon']
    
    leader = arenaData['arenaLeader']
    leaderPoke = []
    for pokemon in leader.team:
        pokemon = pokemon[0]
        leaderPoke.append(pokemon.name)
    
    ArenaLeader.append([leader.name, leader.averageLevel, leaderPoke ])
    ArenaLeaderTable =  pd.DataFrame(ArenaLeader, columns = ArenaLeaderColumnNames)
    # ----- #
    
    # EXTRACT LeaderPokemon DataFrame
    leaderPokemon = []
    leaderPokemonColumnName = ['name', 'moves', 'leaderName']
    
    for pokemon in arenaData['arenaLeader'].team:
        pokemon = pokemon[0]
        leaderName = arenaData['arenaLeader'].name
        myPokemons = []
        moves = []
        for move in pokemon.moves:
            moves.append(move['name'])
        myPokemon = [pokemon.name, moves, leaderName]
        leaderPokemon.append(myPokemon)
    leaderPokemonTable =  pd.DataFrame(leaderPokemon, columns = leaderPokemonColumnName)
    # ----- #
    
    
    
    # write data to CSV
    areaTable.to_csv(f"{pathToSaveCSV}/Areas.csv") 
    myPokemonTable.to_csv(f"{pathToSaveCSV}/EncounterablePokemon.csv") 
    movesOfInterestTable.to_csv(f"{pathToSaveCSV}/MovesOfInterest.csv") 
    ArenaLeaderTable.to_csv(f"{pathToSaveCSV}/ArenaLeader.csv") 
    leaderPokemonTable.to_csv(f"{pathToSaveCSV}/LeaderPokemon.csv") 
    # ----- #